# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [4]:
# your code here
orders=pd.read_csv('orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [7]:
# your code here
#Sub Problem 1:
customer_data = orders.groupby('CustomerID').agg(total_spent=('amount_spent', 'sum')) 
customer_data

,total_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [8]:
# Calculate the 0.95 and 0.75 quantiles of the 'total_spent' column
quantiles = customer_data['total_spent'].quantile([0.95, 0.75])
quantiles

0.95    5840.182
0.75    1661.640
Name: total_spent, dtype: float64

In [10]:
labels=['Regular','Preferred','VIP']
customer_data['customer_type'] = pd.cut(customer_data['total_spent'], bins=[0,quantiles[0.75], quantiles[0.95],100000], labels=labels)
customer_data

vip_customer = customer_data[customer_data['customer_type']=='VIP']
preferred_customer = customer_data[customer_data['customer_type']=='Preferred']

vip_customer
preferred_customer

,total_spent,customer_type
CustomerID,,
12347,4310.00,Preferred
12348,1797.24,Preferred
12349,1757.55,Preferred
12352,2506.04,Preferred
12356,2811.43,Preferred
...,...,...
18259,2338.60,Preferred
18260,2643.20,Preferred
18272,3078.58,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [12]:
# your code here
vip_country = orders[orders['CustomerID'].isin(vip_customer.index)].groupby(['CustomerID','Country']).agg({'amount_spent':sum}).sort_values('amount_spent', ascending=False)
vip_country 

,,amount_spent
CustomerID,Country,
17511,United Kingdom,91062.38
16029,United Kingdom,81024.84
12346,United Kingdom,77183.60
16684,United Kingdom,66653.56
14096,United Kingdom,65164.79
...,...,...
14944,United Kingdom,5900.25
13209,United Kingdom,5891.61
13534,United Kingdom,5870.08


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [14]:
# your code here
preferred_country = orders[orders['CustomerID'].isin(preferred_customer.index)].groupby(['CustomerID','Country']).agg({'amount_spent':sum}).sort_values('amount_spent', ascending=False)
preferred_country

,,amount_spent
CustomerID,Country,
13050,United Kingdom,5836.86
12720,Germany,5781.73
15218,United Kingdom,5756.89
17686,United Kingdom,5739.46
13178,United Kingdom,5725.47
...,...,...
12455,Spain,767.96
12429,Austria,437.98
12417,Spain,436.30
